# Styling the page



# Adding Google OAuth

## Configuring Google Cloud

1. Create a new project in https://console.cloud.google.com/
2. Configure consent screen.
    1. Select external mode
    2. Fill out app name, user support email, and developer contact info.
    3. Skip scopes. scopes are not needed for simple authentication.
    4. Add test users
3. Go to credentials -> create credentials -> create OAuth client id
    1. Add both http://localhost and http://localhost:3000 for both the authorized JavaScript Origins and Authorized redirect URIs.

## Configuring React 

1. Go into `index.html` and add this script: 
```html
<script src="https://accounts.google.com/gsi/client" defer async></script>
```

The script above allows you to to get access to implementing google variavles in JavaScript and HTML. It gives access to the global `google` variable in JS.


### Creating environment variables

Create environment variables in a `.env` file. Make sure to prefix all variables in an env file with the `REACT_APP` prefix, so that react will know that it should use those environment variables. 

- Variables can be directly accessed through the `process.env` variable.
- Whenever you add new environment variables, you must restart your server to reflect the changes.

1. Create a `.env` file 
2. Add variables and prefix with `REACT_APP`. 
3. Restart server.

In [ ]:
REACT_APP_CLIENT_ID = 212521233525-0p0onok0cqtvpal0130chl7hq8viltqp.apps.googleusercontent.com
REACT_APP_CLIENT_SECRET = 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

### Adding google button

To add a sign in google button, the process is defined by these three steps: 

1. Add the `/* global google */` comment so that you get access to the `google` object.
1. initialize the sign-in process by providing the client id and what happens after the user signs in by clicking on the button. 
2. Render the button by accessing an element and applying the styling.
3. Optionally add the one-tap sign in prompt.

> initialization

```javascript
google.accounts.id.initialize({
      client_id: process.env.REACT_APP_CLIENT_ID,
      callback: handleCredentialResponse,
});
```

Using the `google.accounts.id.initialize()` function, pass in an object with two keys: 

- `client_id` : the google oauth client ID that you got from the google cloud setup. 
- `callback` : a function that takes in a `response` object, which represents the succussful user login transaction.

This sets up the entire sign-in process. All sign-in methods will employ the client id and the specified callback. 

> Button rendering

```js
google.accounts.id.renderButton(
        document.getElementById("buttonDiv"),
        { theme: "outline", size: "large" } 
);
```

The `google.accounts.id.renderButton()` function takes in two arguments: 

1. The element to choose to style as the google button. This should be an empty div. 
2. An object that describes how google shoudl style the button.

> Google one-tap-sign-in 

```js
google.accounts.id.prompt();  
```

This simple function will show the one-tap-sign-in prompt, which is still linked to the same callback that you defined in the prior initialization.

> callback 

```js
const handleCredentialResponse = (response) => {
    console.log("Encoded JWT ID token: " + response.credential);
    const user = jwtDecode(response.credential);
};
```

- The callback runs after the user successfully signs in, and accepts a `response` parameter.
- The JWT containing the user data is stored on `response.credential`
- Install a JWT decoder using `npm i jwt-decode` and decode the token using the `jwtDecode()` function. This returns a user object.
      ```js
      import jwtDecode from "jwt-decode";
      ```

In [ ]:
const handleCredentialResponse = (response) => {
    console.log("Encoded JWT ID token: " + response.credential);
    const { email, name, picture } = jwtDecode(response.credential);
    login({ email, name, picture });
  };

In [ ]:
useEffect(() => {
    /* global google */
    
    google.accounts.id.initialize({
      client_id: process.env.REACT_APP_CLIENT_ID,
      callback: handleCredentialResponse,
    });
    
      google.accounts.id.renderButton(
        document.getElementById("buttonDiv"),
        { theme: "outline", size: "large" } 
      );
      google.accounts.id.prompt();  
  });

### Handling login logout flow

The easiest way to handle auth flow is to have a global context dedicated for authentication. The main idea is that when the `user` is null, you are logged out, and when the `user` is not null, you are logged in. There are three parts to the context: 

1. user state 
2. login method 
3. logout method 

> user state 

The user is the information we get back from the decoded jwt token. Else, when are not logged in, it is null. 

- When the user is logged in (not null), show the logout button.
- When the user is logged out (null), do not show the progress section. 

> login

The purpose of the login method is to set the user and to hide the sign-in button.

- The login method takes in information and sets the user using `setUser()`. 
- Hide the sign-in button using `document.getElementById("buttonDiv").hidden = true`.

> logout 

The purpose of the logout method is to set the user to null and to show the sign-in button.

- Do `setUser(null)` 
- Show the sign-in button using `document.getElementById("buttonDiv").hidden = true`. 

In [ ]:
import React from "react";
export const AuthContext = React.createContext({});

export const AuthContextProvider = (props) => {
  const [user, setUser] = React.useState(null);

  const login = ({ email, name, picture }) => {
    setUser({ email: email.substring(0, email.indexOf("@")), name, picture });
    document.getElementById("buttonDiv").hidden = true;
  };

  const logout = () => {
    setUser(null);
    document.getElementById("buttonDiv").hidden = false;
  };

  return (
    <AuthContext.Provider value={{ user, login, logout }}>
      {props.children}
    </AuthContext.Provider>
  );
};

# Making a modal

## Modal structure overview

1. Create a modal component that renders using react portals. 
2. Have the modal component take in a state prop that controls its visibility. 
3. style the modal and modal overlay with a high Z-index so that they appear above all elements.

## Modal logic 

We can control the visibility of the modal using boolean state. We make the modal accept two props: 

1. The `isOpen` prop, which is the boolean state of the modal visibility. 
2. The `onCloseModal` prop, which is a function that sets the state of modal visibility to false and thus closes the modal. 

- When the `modalIsOpen` prop is true, our modal should appear, and when it is false, it should be hidden. 
- Elsewhere in our code, like on a button, we can add the function to open the modal by changing the visibility state to true.

In [ ]:
const Dummy = () => {
    const [modalIsOpen, setModalIsOpen] = useState(false);
    return (
        <>
            <Modal
            isOpen={modalIsOpen}
            onCloseModal={() => setModalIsOpen(false)}
            />
            <MyContent>
                <OpenModalButton onClick={() => setModalIsOpen(true)}/> 
            </MyContent>
        </>
    );
}

## Using React portals 

React portals are a way of rendering HTML content outside of the root div in `index.html`. We need React Portals for modals because due to the CSS laws, any subcontent inside an element is subject to that same parent's stacking index. What that means is that if you're rendering a modal inside some parent element with a Z-index of 0, then no matter what, that modal will never have a Z-index higher than 0. 

```js
ReactDOM.createPortal(<ComponentToRender /> , elementToRenderIn)
```

The `ReactDOM.createPortal()` method takes in two arguments: 

1. The component to render 
2. The HTML element to render the component in. For this, go to `index.html` and manually add some `<div>` outside of the root div.

In [ ]:
import ReactDOM from "react-dom";

const Modal = ({ isOpen, onCloseModal }) => {
    if (!isOpen) {
      return null;
    }
  
    return (
      <>
        {ReactDOM.createPortal(
          <ModalContent onCloseModal={onCloseModal} />,
          document.querySelector("#overlay-root")
        )}
      </>
    );
  };

## Modal Content Structure. 

```html
<React.Fragment>
    <div className="modal-overlay"></div>
    <div className="modal">
        <SomeContent />
    </div>
<React.Fragment/>
```

You are going to render two things: the modal overlay and the modal. 

- The modal overlay is the darkened background while the modal in on screen, signaling that the modal is the only active thing on the screen.
- The modal is the actual modal.

In [ ]:
const ModalContent = ({ onCloseModal }) => {
  
    return (
      <>
        <div className="modal-overlay"></div>
        <div className="modal">
          <SomeContent />
        </div>
      </>
    );
  };

## Modal styling 

This is the styling you have to apply to both the modal and modal overlay for them to work correctly.

```css
.modal {
  position: fixed;
  top: 50%;
  left: 50%;
  transform: translate(-50%, -50%);
  background-color: gray;
  padding: 3rem;
  z-index: 1000;
}
```

```css
.modal-overlay {
  position: fixed;
  top: 0;
  left: 0;
  right: 0;
  bottom: 0;
  z-index: 999; 
  background-color: rgba($color: #000000, $alpha: 0.6);
}
```

# Storing and handling data with Firebase

## Firebase setup

1. Create a new project in firebase 
2. Create a realtime database in test mode, where anyone can read or write. 
3. Copy the realtime database URL

## Realtime Database Overview 

The entire structure is based on your url structure for API requests. Any subroute that you do becomes a collection, and with more nested routes, you can get more collections within collections. The collection that you create will end with a `.json`. Each piece of data in a collection is bound by a **firebase name**, which is a hashed ID.

![](2022-10-12-20-20-24.png)

- Collections are referenced by the route to get there, and then the collection name and `.json`. 
- Individual documents are referenced by the firebasename and then .json within the collection.

> route structure 

```js
const url = "https://hours-896df-default-rtdb.firebaseio.com/";
const url1 = `${url}/goals.json`
const url2 = `${url}/goals/${firebasename}.json` 
```

- `url1` references the goals collection 
- `url2` references a specific document within the goals collection.

> gotcha to look out for 

You cannot use emails as a collection name. Any collection name with a `.` or an `@` in it is not a valid collection name and will cause errors in the API requests.


In [ ]:
const url = "https://hours-896df-default-rtdb.firebaseio.com/";

`${url}/goals/goal1.json`  // * references the goal1 collection in the goals collection.

## Fetching data

To fetch all data from a collection, make a **GET** request to the specified collection url. The data returned is in the shape of a dictionary, with the keys being the firebase names, so you have to loop through the dictionary to access the values.

![](2022-10-12-20-32-45.png)

In [ ]:
const fetchCards = async (email) => {
    const response = await fetch(`${url}/goals/${email}.json`);
    const data = await response.json();
    const temp = [];
    for (const key in data) {
      temp.push(data[key]);
    }
    return temp;
  };

## Posting data

To add data to a collection, simply make a **POST** request to the specified collection you want. Make sure that you are sending JSON data, however. 

1. Convert anything sent to JSON 
2. Add the `"Content-Type": "application/json"` headers to ensure JSON format for firebase.

In [ ]:
const postCards = ({ currentHours, goalHours, title, id }, email) => {
    fetch(`${url}/goals/${email}.json`, {
      method: "POST",
      body: JSON.stringify({ currentHours, goalHours, title, id }),
      headers: {
        "Content-Type": "application/json",
      },
    })
      .then((response) => response.json())
  };

## Updating data

To update data at a specific document, make sure you send a **PUT** request to the specified document url. Put the updated data in the body, which should be your entire data schema.

In [ ]:
const updateCard = async (id, payload, email) => {
    const { hoursWorked, minutesWorked} = payload;
    const { firebaseName, card } = await findCardFromId(id, email);  // find the specified data and the firebase name associated with it.
    card.currentHours += hoursWorked + Number((minutesWorked / 60).toFixed(1));
    await fetch(`${url}/goals/${email}/${firebaseName}.json`, {
      method: "PUT",
      body: JSON.stringify({ ...card }),
    });
    return card;
  };

## Deleting data

To delete a specified document node, simply make a **DELETE** request to document url.

In [ ]:
const deleteCards = async (id, email) => {
    const { firebaseName } = await findCardFromId(id, email);
    await fetch(`${url}/goals/${email}/${firebaseName}.json`, {
      method: "DELETE",
    });
    return id;
  };

# Various

## Creating a progress bar

```scss
.progress {
      height: 20px;
      border-radius: 12px;
      background-color: black;
      padding-left: 1rem;
      display: flex;
      align-items: center;
      border: 2px dotted $primary-color;
      div {
        background-color: red;
        border-radius: 12px;
      }
    }
```

In [ ]:
const ProgressBar = ({ currentHours, goalHours }) => {
    return (
      <div className="progress">
        <div
          style={{ width: `${(currentHours / goalHours) * 100}%` }} 
        >
          {(currentHours / goalHours) * 100}%
        </div>
      </div>
    );
  };